# ACI Create New User

This example shows how to create a new controller user.

We need to follow a workflow to gather the info needed for the POST request.
Additionally, each input needs to be validated before posting, else we need to capture server responses and handle them correctly. 

Things we should do before requesting input:
 - Get all users from APIC.
 - Get all roles from APIC.
 - Get all security domains from APIC.

When we take input, we can then present a list of roles and domains to select from, rather than allowing free-form typing.

In [1]:
# Set up all the modules we need.
import requests
import json
import pandas as pd
import os
import getpass
import sys

In [2]:
def get_info():
    # Read controller host/IP and auth credentials from a config file, if available. 
    # Test for config.py and import the variables.
    # Check if the variable is imported; if not, prompt for input.
    if os.path.isfile('./config.py'):
        from config import controller, username, password

    try:
        controller
    except NameError:
        controller = input("Controller Hostname/IP: ")

    try:
        username
    except NameError:
        username = input("APIC Username: ")
    
    try: 
        password 
    except NameError:
        password = getpass.getpass("APIC Password: ")

    return controller, username, password

In [3]:
controller, username, password = get_info()

In [4]:
print("Controller: " + controller)
print("Username: " + username)
#print("Password: " + password)

Controller: 10.18.188.101
Username: api-andrew


In [5]:
# https://stackoverflow.com/questions/26310467/python-requests-keep-session-between-function
def session_init():
    # Set up the request and pass the authentication data as JSON
    s = requests.session()
    return s

In [6]:
def auth(controller, username, password):
    # Shhh! We don't want self-signed cert warnings.
    #requests.packages.urllib3.disable_warnings() 
    # Set up the POST request and pass the authentication data as JSON
    auth_class = "aaaLogin.json"
    auth_url = base_url + auth_class
    # Setup auth data as a dictionary 
    auth_data = {
        "aaaUser":{
            "attributes":{
                "name":username,
                "pwd":password
            }
        }
    }
    response = s.post(auth_url, json=auth_data, verify=False)
    return response

In [7]:
# Set up our URL
base_url = "https://" + str(controller) + "/api/"

#print(auth_url)
#print(auth_data)

In [8]:
s = session_init()
s
reply = auth(controller, username, password)
reply.status_code

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

In [9]:
# We can use this to test the reponse code to determine if login was successful,
# and do something specific based on that (like execute a function).
#
# This can be expanded greatly to test against different codes (403, 404, 500, etc)
# if we wanted.

if reply.status_code != 200:
    print("Didn't authenticate.")
    print("HTTP Status: " + str(response))
else:
    print("All good")

All good


In [10]:
# Set up our user class URLs.
# We set up one for GETting data (inlcuding child objects)
# and one for POSTing data.

qstring = "?rsp-subtree=children"
# Construct the user post URL
user_class="node/mo/uni/userext.json"
post_user_url = base_url + user_class
get_user_url = base_url + user_class  + qstring
print(get_user_url)
print(post_user_url)

https://10.18.188.101/api/node/mo/uni/userext.json?rsp-subtree=children
https://10.18.188.101/api/node/mo/uni/userext.json


In [11]:
def get_users():
    users = s.get(get_user_url, verify=False)
    users = users.json()
    print(json.dumps(users, indent=4, sort_keys=True))
    dn = users['imdata'][0]['aaaUserEp']['attributes']['dn']
    name_list = []
    firstname_list = []
    lastname_list = []
    status_list = []
    phone_list = []
    users_list = users['imdata'][0]['aaaUserEp']['children']
    for user in users_list:
        if "aaaUser" in user: 
            #print(user)
            rn = user['aaaUser']['attributes']['rn']
            name = user['aaaUser']['attributes']['name']  
            firstname = user['aaaUser']['attributes']['firstName']
            lastname = user['aaaUser']['attributes']['lastName']
            status = user['aaaUser']['attributes']['accountStatus']
            phone = user['aaaUser']['attributes']['phone']
            name_list.append(name)
            firstname_list.append(firstname)
            lastname_list.append(lastname)
            phone_list.append(phone)
            status_list.append(status)
    zlist = zip(status_list, name_list, firstname_list, lastname_list, phone_list, rn_list)
    df_input = list(zlist)
    df = pd.DataFrame(df_input, columns=("Status","User Name","First Name","Last Name","Phone","RN"))

In [12]:
def get_roles():
    roles = [1, 2, 3]
    return roles

In [13]:
def get_domains():
    domains = [1,2,3]
    return domains

In [14]:
def get_new_user():
    os.system('cls' if os.name == 'nt' else 'clear')
    prompt = input("Add New User? (y/N)")
    if prompt.lower() in ('n', 'N'):e
        sys.exit()
    elif prompt.lower() in ('y', 'Y'):
        get_user_input()
    else:
        get_new_user()

In [15]:
def input_pass():
    password1 = getpass.getpass("[Required] Password: ")
    password2 = getpass.getpass("[Required] Re-enter Password: ")
    if password1 != password2:
        print("Passwords are not the same. DO OVER!")
        print("(unless you wanna bug out now)")
        do_over = input("[D]o over, or [E]xit?")
        if do_over in ('D','d'):
            input_pass()
        sys.exit()
    return password1

In [16]:
def get_user_input():
    print("There's not much error checking here.") 
    print("You'll get a chance to re-enter, or you can just stop/restart the program.")
    print("Enter accepts [default] option.")
    print("--------------------")
    user = input("Username: ")
    print(" ")
    print("Password needs three of the following: ")
    print(" - lower case letter")
    print(" - upper case letter")
    print(" - number")
    print(" - symbol")
    print(" ")
    passwd = input_pass()
    firstname = input("[Optional] First Name: ")
    lastname = input("[Optional] Last Name: ")
    phone = input("[Optional] Phone: ")
    email = input("[Optional] Email: ")
    print("Enter security domains, with a [Y]es or [N]o: ")
    domain_all = input("Security Domain - All?  (y/n): ")
  
    if domain_all.lower() in ('y','Y','Yes','YES'):
        domain_all_priv = input("[R]ead or [W]rite: ")
    else:
        domain_all_priv = ""

    domain_common = input("Security Domain - Common? (y/n): ")
    if domain_common.lower() in ('y','Y','Yes','YES'):
        domain_common_priv = input("[R]ead or [W]rite: ")
    else:
        domain_common_priv = ""

    domain_mgmt = input("Security Domain - Mgmt? (y/n): ")
    if domain_mgmt.lower() in ('y','Y','Yes','YES'):
        domain_mgmt_priv = input("[R]ead or [W]rite: ")
    else:
        domain_mgmt_priv = ""

    expires = input("Expires? [y/n]")
    activate = input("Activate User? [y/n]")
    print("----------------------------------------")
    print("How does this look?")
    print("Username: " + str(user))
    print("Password: ***** ")
    print("First Name: " + str(firstname))
    print("Last Name: " + str(lastname))
    print("Email: " + str(phone))
    print("Email: " + str(email))
    print("Security Domain - All: " + str(domain_all))
    print("Security Domain - All: " + str(domain_all_priv))
    print("Security Domain - Common: " + str(domain_common))
    print("Security Domain - Common: " + str(domain_common_priv))
    print("Security Domain - Mgmt: " + str(domain_mgmt))
    print("Security Domain - Mgmt: " + str(domain_mgmt_priv))
    print("Expires: " + str(expires))
    print("Active: " + str(activate))
    is_good = input("[G]o go gadget FORM POST!" + "\n" + "[D]o over. *sad trombone")
    if is_good in ('G','g'):
        print("An API POST happens here.")
    elif is_good in ('D','d'):
        
    return user, passwd, firstname, lastname, phone, email, domain_all, domain_all_priv, domain_common, domain_common_priv, domain_mgmt, domain_mgmt_priv, expires, activate 

In [17]:
get_roles()

[1, 2, 3]

In [18]:
get_domains()

[1, 2, 3]

In [19]:
get_users()

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{
    "imdata": [
        {
            "aaaUserEp": {
                "attributes": {
                    "annotation": "",
                    "childAction": "",
                    "descr": "",
                    "dn": "uni/userext",
                    "extMngdBy": "",
                    "lcOwn": "local",
                    "modTs": "2018-11-17T13:52:59.962-05:00",
                    "monPolDn": "uni/fabric/monfab-default",
                    "name": "",
                    "nameAlias": "",
                    "newSubscr": "yes",
                    "ownerKey": "",
                    "ownerTag": "",
                    "pwdStrengthCheck": "yes",
                    "status": "",
                    "uid": "0"
                },
                "children": [
                    {
                        "aaaUserConf": {
                            "attributes": {
                                "annotation": "",
                                "childAction": "deleteNonPresent"

NameError: name 'rn_list' is not defined

In [19]:
# Prompt for user info
prompt = input("Add New User? (y/N)")

Add New User? (y/N)y


In [20]:
if prompt.lower() not in ('y', 'Y'):
    sys.exit()

In [21]:
get_user_input()

There's not much error checking here.
You'll get a chance to re-enter, or you can just stop/restart the program.
Enter accepts [default] option.
 
Username: schmuck
 
Password needs three of the following: 
 - lower case letter
 - upper case letter
 - number
 - symbol
 
[Required] Password: ········
[Required] Re-enter Password: ········
[Optional] First Name: asdf
[Optional] Last Name: qwer
[Optional] Phone: asdf
[Optional] Email: 
Enter security domains, with a [Y]es or [N]o: 
Security Domain - All?  (y/n): y
[R]ead or [W]rite: w
Security Domain - Common? (y/n): n
Security Domain - Mgmt? (y/n): n
Expires? [y/n]n
Activate User? [y/n]y


('schmuck',
 'yadda',
 'asdf',
 'qwer',
 'asdf',
 '',
 'y',
 'w',
 'n',
 '',
 'n',
 '',
 'n',
 'y')

In [22]:
print("Username: " + str(user))
print("Password: ***** ")
print("First Name: " + str(firstname))
print("Last Name: " + str(lastname))
print("Email: " + str(phone))
print("Email: " + str(email))
print("Security Domain - All: " + str(domain_all))
print("Security Domain - All: " + str(domain_all_priv))
print("Security Domain - Common: " + str(domain_common))
print("Security Domain - Common: " + str(domain_common_priv))
print("Security Domain - Mgmt: " + str(domain_mgmt))
print("Security Domain - Mgmt: " + str(domain_mgmt_priv))
print("Expires: " + str(expires))
print("Active: " + str(activate))

NameError: name 'user' is not defined

In [ ]:
userext = "uni/userext.json"

## User POST Data
```
{
            "aaaUser": {
                "attributes": {
                    "name": user,
                    "firstName": firstname,
                    "lastName": lastname,
                    "phone": phone,
                    "email": email,
                },
                "children": [{
                    "aaaUserCert": {
                        "attributes": {
                            "name": "userabc.crt",
                            "data": "-----BEGIN CERTIFICATE-----\nMIICjjCCAfegAwIBAgIJAMQnbE <snipped content> ==\n-----END CERTIFICATE-----",
                        },
                        "children": []
                    },
                    "aaaUserDomain": {
                        "attributes": {
                            "name": "all",
                        },
                        "children": [{
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "aaa",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "access-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "fabric-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "nw-svc-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "ops",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "read-all",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "tenant-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "tenant-ext-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "vmm-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }]
                    }
                }]
            }
        }
```